In [ ]:
import json
import os
from tqdm import tqdm
import csv
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.getcwd()

'/content'

In [ ]:
interim_path = os.path.join('drive','My Drive','Colab Notebooks', 'Interim')
raw_path = os.path.join('drive','My Drive','Colab Notebooks','Raw')
if not os.path.exists(interim_path):
  os.makedirs(interim_path)

files = os.listdir(interim_path)
print(files)

['collected.jsonl', 'cleaned.jsonl']


In [ ]:
def remove(x: str) -> str:
    """A function that cleans a give string by removing multiple punctuation, 
    newlines, emojis, urls, and etceteras."""

    punct = list(":;,.!$%^&#*()\"\'[]{}")
    punct += ['~',' | ']
    
    x = demoji(x)
    if '~' in x:
        x = x.replace('~','')
    x = x.replace('\n',' ').replace('...','').split(' ')
    x = ' '.join([i for i in x if not i.startswith('http')])
    x = x.replace('  ',' ').lower()
    for p in punct:
        x = x.replace(p, '')
    x = x.strip()
    return x

def demoji(text:str) -> str:
    regrex_pattern = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251"
    "\u2010-\u201f"
    "\u2000-\u200f"
    "]+"
    )
    return regrex_pattern.sub(r'',text)

def dehash(x: dict) -> str:
    """A function that takes a """
    y = ','.join([i['text'].lower() for i in x])
    return y

In [ ]:
# get the path for the collected data 
collected_path = os.path.join(interim_path, 'collected.jsonl')

# get the path for the new clean data to be written to
clean_path = os.path.join(interim_path, 'cleaned.jsonl')

states_path = os.path.join(raw_path,'states.csv')

state_list = []
with open(states_path, 'r') as states:
    csvReader = csv.reader(states)
    next(csvReader)
    for line in csvReader:
        state_list.append(line[:2])

# instantiate cleaned.jsonl
with open(clean_path, 'w') as clean:

    # open the collected.jsonl
    with open(collected_path, 'r') as file:
        
        lines = file.readlines()
        # iterate over each line
        for l in tqdm(range(len(lines))):
            
            line = lines[l]
            
            # parse each line to be a dictionary with .loads()
            j_line = json.loads(line)

            # remove any line that does not have a location
            if j_line['location'] != "":

                for state in state_list:
                    if state[0] in j_line['location'] or state[1] in j_line['location'] and 'D.C.' not in j_line['location'] and 'DC' not in j_line['location']:
                        j_line['text'] = remove(j_line['text'])

                        # extract and lower the hashtags from list of dictionaries of hashtags
                        j_line['hashtags'] = dehash(j_line['hashtags'])
                        
                        j_line['standardized_loc'] = state[0]

                        # write line to the clean.jsonl file
                        json.dump(j_line, clean)
                        clean.write('\n')
                        
                        break

100%|██████████| 233618/233618 [00:12<00:00, 18192.78it/s]
